In [45]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Now we load in the data file.

In [47]:
original_data = pd.read_csv("../Data/Historical Lead Records - Condensed REMAPPED.csv",
                        encoding="utf-8")
# Need encoding change for weird characters to come through
lead_data = original_data.copy()
lead_data.head(20)

,Title,Job Function,Job Role,Job Level
0,MANAGER-CYBERSECURITY,IT,INFORMATION SECURITY,MANAGER
1,"MANAGER, INFORMATION SECURITY",IT,INFORMATION SECURITY,MANAGER
2,USER EXPERIENCE ANALYST,ENGINEERING,DEVELOPMENT,CONTRIBUTOR
3,NETWORK SPECIALIST,IT,NETWORKING,CONTRIBUTOR
4,DIRECTOR OF PRIVACY AND COMPLIANCE,IT,INFORMATION SECURITY,DIRECTOR
5,"ASSISTANT VP, NETWORK ARCHITECTURE CCIE 1676",IT,NETWORKING,CONTRIBUTOR
6,"DIRECTOR, INFORMATION SECURITY",IT,NETWORKING,DIRECTOR
7,CIO,IT,INFORMATION SECURITY,C-LEVEL
8,DIRECTOR CLOUD OPERATIONS,IT,IT GENERAL,DIRECTOR
9,"DIRECTOR, ENTERPRISE APPLICATIONS",IT,NETWORKING,DIRECTOR


Dimensions of data

In [48]:
lead_data.shape

(865671, 4)

Unique values count

In [49]:
lead_data.nunique()

Title           179398
Job Function        36
Job Role            24
Job Level           21
dtype: int64

Detailed summary

In [50]:
pd.set_option('display.max_rows',100)
for col in lead_data.drop('Title',axis = 1):
    lead_data[col].value_counts()

Job Function
IT                                       601005
ENGINEERING                               80188
UNKNOWN                                   36795
SALES                                     35243
MANAGEMENT                                32227
SUPPORT                                   16454
OPERATIONS                                12860
FINANCE                                   10717
MARKETING                                 10537
SERVICES                                   5268
EDUCATION                                  3059
PUBLIC SECTOR                              2905
HUMAN RESOURCE                             2788
ADMINISTRATION                             1985
CORPORATE                                  1920
LEGAL                                      1613
FACILITIES                                 1535
PURCHASING                                 1287
MEDICAL                                    1266
PROCUREMENT                                1241
OTHER                      

Job Role
INFORMATION SECURITY                   275444
NETWORKING                             218787
NONE TECHNICAL                         165854
DEVELOPMENT                             82743
IT GENERAL                              45789
GOVERNANCE RISK COMPLIANCE              20641
PROGRAM MANAGEMENT                       9913
HELP DESK                                9157
NON-IT                                   8278
BUSINESS SYSTEMS                         7690
SYSTEMS                                  6947
OPERATIONS                               3906
DATA                                     3401
COMMUNICATIONS                           1355
INTEGRATION                               591
IT FACILITIES                             519
VENDOR MANAGEMENT                         132
TRAINING                                  125
BUSINESS CONTINUITY                        80
MANAGER, AD SYSTEMS                         3
SENIOR MANAGER, CRITICAL FACILITIES         2
OTHER                    

Job Level
CONTRIBUTOR                              310984
MANAGER                                  175557
DIRECTOR                                 142587
EXECUTIVE                                110613
C-LEVEL                                   97147
UNKNOWN                                    4558
NON-MANAGER                                1561
VP-LEVEL                                    260
TEAM LEAD                                    74
VP/DIRECTOR                                  61
ENGINEER/ADMIN                               32
CXO                                          28
DIRECTOR LEVEL                               11
DIRECTOR / C-LEVEL                            7
VP                                            6
C-SUITE                                       2
MANAGEMENT                                    1
DIRECTOR OF ENTERPRISE CLOUD BUSINESS         1
ENGINEER                                      1
OTHER                                         1
ADMIN                         

In [51]:
pd.set_option('display.max_rows',100)
for col in lead_data.drop('Title',axis = 1):
    temp = lead_data[col].value_counts(normalize=True)
    temp
    temp.iloc[0:5].sum()

Job Function
IT                                       0.697221
ENGINEERING                              0.093025
UNKNOWN                                  0.042686
SALES                                    0.040885
MANAGEMENT                               0.037386
SUPPORT                                  0.019088
OPERATIONS                               0.014919
FINANCE                                  0.012433
MARKETING                                0.012224
SERVICES                                 0.006111
EDUCATION                                0.003549
PUBLIC SECTOR                            0.003370
HUMAN RESOURCE                           0.003234
ADMINISTRATION                           0.002303
CORPORATE                                0.002227
LEGAL                                    0.001871
FACILITIES                               0.001781
PURCHASING                               0.001493
MEDICAL                                  0.001469
PROCUREMENT                          

0.911203119253922

Job Role
INFORMATION SECURITY                   0.319778
NETWORKING                             0.254002
NONE TECHNICAL                         0.192549
DEVELOPMENT                            0.096061
IT GENERAL                             0.053159
GOVERNANCE RISK COMPLIANCE             0.023963
PROGRAM MANAGEMENT                     0.011509
HELP DESK                              0.010631
NON-IT                                 0.009610
BUSINESS SYSTEMS                       0.008928
SYSTEMS                                0.008065
OPERATIONS                             0.004535
DATA                                   0.003948
COMMUNICATIONS                         0.001573
INTEGRATION                            0.000686
IT FACILITIES                          0.000603
VENDOR MANAGEMENT                      0.000153
TRAINING                               0.000145
BUSINESS CONTINUITY                    0.000093
MANAGER, AD SYSTEMS                    0.000003
SENIOR MANAGER, CRITICAL FACILI

0.9155476043145672

Job Level
CONTRIBUTOR                              0.368686
MANAGER                                  0.208131
DIRECTOR                                 0.169043
EXECUTIVE                                0.131137
C-LEVEL                                  0.115172
UNKNOWN                                  0.005404
NON-MANAGER                              0.001851
VP-LEVEL                                 0.000308
TEAM LEAD                                0.000088
VP/DIRECTOR                              0.000072
ENGINEER/ADMIN                           0.000038
CXO                                      0.000033
DIRECTOR LEVEL                           0.000013
DIRECTOR / C-LEVEL                       0.000008
VP                                       0.000007
C-SUITE                                  0.000002
MANAGEMENT                               0.000001
DIRECTOR OF ENTERPRISE CLOUD BUSINESS    0.000001
ENGINEER                                 0.000001
OTHER                                   

0.9921694667294215

A few directives to clean this data:

1. Job roles to remain - Information Security (group in 'INformation Security', 'information security'), Networking (group in 'Netoworking'), IT General (group in 'IT Facilities', 'IT', 'Senior Manager, Information Technology'), Development, Systems (Group in 'Business Systems'), Governance Risk Compliance (not indicated but there are a significant number of them, group in 'Senior Manager, Security, Risk, and Compliance', 'IT/IS Compliance/Risk/Control Staff'); everything else will flow through as Non-ICP
2. Job functions to remain - IT (group in 'Information Technology','IT - Security','IT - Network','Information Security, Information Technology','IT Operations','IT-Sec Admin','Director Global IT','Information Security, Information Technology, Enterprise Architecture','It','Information Technology, Information Technology Executive'), Engineering (include 'Engineering & Technical','Engineer SASE'), Procurement (group in 'Purchasing','Sourcing / Procurement'), Risk/Legal/Compliance (include 'Legal','Risk, Legal Operations','Lawyer / Attorney','Governmental Affairs & Regulatory Law')
3. Job levels to remain - Contributor (include 'Individual Contributor','contributor','contribtuor'), Manager (include 'Management','Manager Level','manager','Threat Hunting Manager','IT Security Manager'), Executive (include 'Senior Executive','Exec.'), Director (include 'Director Level','IT Infrastructure Director','Director of Enterprise Cloud Business','IT Security Director'), C-Level (group in 'C-level','CxO','C level','C-suite','Director (It & Project) & Chief Information Security Officer','C Level')

In [54]:
lead_data = lead_data.replace({'Job Role':['NETOWORKING']}, 'NETWORKING')
lead_data = lead_data.replace({'Job Role':['IT FACILITIES', 'IT', 'SENIOR MANAGER, INFORMATION TECHNOLOGY']}, 'IT GENERAL')
lead_data = lead_data.replace({'Job Role':['BUSINESS SYSTEMS']}, 'SYSTEMS')
lead_data = lead_data.replace({'Job Role':['SENIOR MANAGER, SECURITY, RISK, AND COMPLIANCE', 'IT/IS COMPLIANCE/RISK/CONTROL STAFF']}, 'GOVERNANCE RISK COMPLIANCE')
lead_data.loc[~lead_data['Job Role'].isin(['INFORMATION SECURITY','NETWORKING','IT GENERAL','SYSTEMS','GOVERNANCE RISK COMPLIANCE']) &
            ~lead_data['Job Role'].isna(),
            lead_data.columns == 'Job Role'] = 'NON-ICP'

In [55]:
lead_data = lead_data.replace({'Job Function':['INFORMATION TECHNOLOGY','IT - SECURITY','IT - NETWORK','INFORMATION SECURITY, INFORMATION TECHNOLOGY','IT OPERATIONS','IT-SEC ADMIN','DIRECTOR GLOBAL IT','INFORMATION SECURITY, INFORMATION TECHNOLOGY, ENTERPRISE ARCHITECTURE','INFORMATION TECHNOLOGY, INFORMATION TECHNOLOGY EXECUTIVE']},
                            'IT')
lead_data = lead_data.replace({'Job Function':['ENGINEERING & TECHNICAL','ENGINEER SASE']},'ENGINEERING')
lead_data = lead_data.replace({'Job Function':['PURCHASING','SOURCING / PROCUREMENT']},'PROCUREMENT')
lead_data = lead_data.replace({'Job Function':['LEGAL','RISK, LEGAL OPERATIONS','LAWYER / ATTORNEY','GOVERNMENTALK AFFAIRS & REGULATORY LAW']},
                            'RISK/LEGAL/COMPLIANCE')
lead_data.loc[~lead_data['Job Function'].isin(['IT','ENGINEERING','PROCUREMENT','RISK/LEGAL/COMPLIANCE']) &
            ~lead_data['Job Function'].isna(),
            lead_data.columns == 'Job Function'] = 'NON-ICP'

In [56]:
lead_data = lead_data.replace({'Job Level':['INDIVIDUAL CONTRIBUTOR','CONTRIBTUOR']},'CONTRIBUTOR')
lead_data = lead_data.replace({'Job Level':['MANAGEMENT','MANAGER LEVEL','MANAGER','THREAT HUNTING MANAGER','IT SECURITY MANAGER']},'MANAGER')
lead_data = lead_data.replace({'Job Level':['SENIOR EXECUTIVE','EXEC.']},'EXECUTIVE')
lead_data = lead_data.replace({'Job Level':['DIRECTOR LEVEL','IT INFRASTRUCTURE DIRECTOR','DIRECTOR OF ENTERPRISE CLOUD BUSINESS','IT SECURITY DIRECTOR']},'DIRECTOR')
lead_data = lead_data.replace({'Job Level':['CXO','C-SUITE','DIRECTOR (IT & PROJECT) & CHIEF INFORMATION SECURITY OFFICER','C LEVEL']},'C-LEVEL')
lead_data.loc[~lead_data['Job Level'].isin(['CONTRIBUTOR','MANAGER','EXECUTIVE','DIRECTOR','C-LEVEL']) &
            ~lead_data['Job Level'].isna(),
            lead_data.columns == 'Job Level'] = 'UNKNOWN'

Now let's look at the counts of the modified dataframe

In [57]:
for col in lead_data.drop('Title',axis = 1):
    lead_data[col].value_counts()

Job Function
IT                       601008
NON-ICP                  176661
ENGINEERING               80190
PROCUREMENT                2528
RISK/LEGAL/COMPLIANCE      1614
Name: count, dtype: int64

Job Role
NON-ICP                       285544
INFORMATION SECURITY          275444
NETWORKING                    218787
IT GENERAL                     46308
GOVERNANCE RISK COMPLIANCE     20641
SYSTEMS                        14637
Name: count, dtype: int64

Job Level
CONTRIBUTOR    310984
MANAGER        175558
DIRECTOR       142599
EXECUTIVE      110613
C-LEVEL         97177
UNKNOWN          6562
Name: count, dtype: int64

In [58]:
for col in lead_data.drop('Title',axis = 1):
    lead_data[col].value_counts(normalize=True)

Job Function
IT                       0.697224
NON-ICP                  0.204943
ENGINEERING              0.093028
PROCUREMENT              0.002933
RISK/LEGAL/COMPLIANCE    0.001872
Name: proportion, dtype: float64

Job Role
NON-ICP                       0.331503
INFORMATION SECURITY          0.319778
NETWORKING                    0.254002
IT GENERAL                    0.053761
GOVERNANCE RISK COMPLIANCE    0.023963
SYSTEMS                       0.016993
Name: proportion, dtype: float64

Job Level
CONTRIBUTOR    0.368686
MANAGER        0.208132
DIRECTOR       0.169058
EXECUTIVE      0.131137
C-LEVEL        0.115208
UNKNOWN        0.007780
Name: proportion, dtype: float64

Number of rows where any one of the outputs are NA

In [59]:
(lead_data['Job Role'].isna()|lead_data['Job Function'].isna()|lead_data['Job Level'].isna()).sum()
(lead_data['Job Role'].isna()|lead_data['Job Function'].isna()|lead_data['Job Level'].isna()).sum()/lead_data.shape[0]

25569

0.02953662534611879

To make modeling easier, it might be acceptable to drop this 3% of the data. Now let's see the number of distinct words in the job title field. Need to filter out NaNs.

In [60]:
lead_data['Title'].isna().sum()
lead_data['Title'].isna().sum()/lead_data.shape[0]

12730

0.014705355729832696

In [61]:
(lead_data['Job Role'].isna()|lead_data['Job Function'].isna()|lead_data['Job Level'].isna()|lead_data['Title'].isna()).sum()
(lead_data['Job Role'].isna()|lead_data['Job Function'].isna()|lead_data['Job Level'].isna()|lead_data['Title'].isna()).sum()/lead_data.shape[0]

27978

0.03231943775406592

Nearly 28K NAs, which is about 3% of the data.

In [62]:
words_series = lead_data['Title'].loc[~lead_data['Title'].isna()].str.lower().str.split().tolist()
words_list = [word for title in words_series for word in title]

Unique number of words

In [63]:
words_set = set(words_list)
len(words_set)

31770

Remove stopwords

In [64]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\csarc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
len(words_set - stop_words)

31667

Not that many overlapping, so there's not really that much of an impact of removing stopwords. Thus, for the model, I won't even bother to do this.